**환경 설정**

In [ ]:
!pip install ipython-sql==0.4.1
!pip install SQLAlchemy==1.4.49

In [3]:
%load_ext sql

## 사용자별로 처음 채널과 마지막 채널 알아내기

In [ ]:
%%sql

WITH first AS (
   SELECT userid, ts, channel, ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts) seq
   FROM raw_data.user_session_channel usc
   JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
), last AS (
   SELECT userid, ts, channel, ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts DESC) seq
   FROM raw_data.user_session_channel usc
   JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
)
SELECT first.userid AS userid, first.channel AS first_channel, last.channel AS last_channel
FROM first
JOIN last ON first.userid = last.userid and last.seq = 1
WHERE first.seq = 1;

In [ ]:
%%sql

SELECT first.userid AS userid, first.channel AS first_channel, last.channel AS last_channel
FROM (
  SELECT userid, ts, channel, ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts) seq
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
) first
JOIN (
  SELECT userid, ts, channel, ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts DESC) seq
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
)  last ON first.userid = last.userid and last.seq = 1
WHERE first.seq = 1;

In [ ]:
%%sql

SELECT userid,
 MAX(CASE WHEN rn1 = 1 THEN channel END) first_touch,
 MAX(CASE WHEN rn2 = 1 THEN channel END) last_touch
FROM (
  SELECT userid,
    channel,
    (ROW_NUMBER() OVER (PARTITION BY usc.userid ORDER BY  st.ts asc)) AS rn1,
    (ROW_NUMBER() OVER (PARTITION BY usc.userid ORDER BY  st.ts desc)) AS rn2
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
)
GROUP BY 1;

In [ ]:
%%sql

SELECT DISTINCT
    A.userid,
    FIRST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS First_Channel,
    LAST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS Last_Channel
FROM raw_data.user_session_channel A
LEFT JOIN raw_data.session_timestamp B ON A.sessionid = B.sessionid
ORDER BY 1;

## Gross Revenue가 가장 큰 UserID 10개 찾기

In [ ]:
%%sql

SELECT
    userID,
    SUM(amount)
FROM raw_data.session_transaction st
LEFT JOIN raw_data.user_session_channel usc ON st.sessionid = usc.sessionid
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

In [ ]:
%%sql

SELECT DISTINCT
    usc.userid,
    SUM(amount) OVER(PARTITION BY usc.userid)
FROM raw_data.user_session_channel AS usc
JOIN raw_data.session_transaction AS revenue ON revenue.sessionid = usc.sessionid
ORDER BY 2 DESC
LIMIT 10;

## raw_data.nps 테이블을 바탕으로 월별 NPS 계산

In [ ]:
%%sql

SELECT month,
  ROUND((promoters-detractors)::float/total_count*100, 2) AS overall_nps
FROM (
  SELECT LEFT(created, 7) AS month,
    COUNT(CASE WHEN score >= 9 THEN 1 END) AS promoters,
    COUNT(CASE WHEN score <= 6 THEN 1 END) AS detractors,
    COUNT(CASE WHEN score > 6 AND score < 9 THEN 1 END) As passives,
    COUNT(1) AS total_count
  FROM raw_data.nps
  GROUP BY 1
  ORDER BY 1
);

In [ ]:
%%sql

SELECT LEFT(created_at, 7) AS month,
  ROUND(SUM(CASE
    WHEN score >= 9 THEN 1
    WHEN score <= 6 THEN -1 END)::float*100/COUNT(1), 2)
FROM raw_data.nps
GROUP BY 1
ORDER BY 1;

## 트랜잭션

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.yeojun_name_gender;
CREATE TABLE adhoc.yeojun_name_gender (
  name varchar(32) primary key,
  gender varchar(16)
);
INSERT INTO adhoc.yeojun_name_gender VALUES ('Ben', 'Male'), ('Maddie', 'Female');

In [ ]:
%%sql

SELECT *
FROM adhoc.yeojun_name_gender;

In [6]:
import psycopg2

# Redshift connection 함수
def get_Redshift_connection(autocommit):
    host = host + "redshift.amazonaws.com"
    redshift_user = userid
    redshift_pass = userpass
    port = portnum
    dbname = databasename
    conn = psycopg2.connect("dbname={dbname} user={user} host={host} password={password} port={port}".format(
        dbname=dbname,
        user=redshift_user,
        password=redshift_pass,
        host=host,
        port=port
    ))
    conn.set_session(autocommit=autocommit)
    return conn

### INSERT SQL을 autocommit=False로 실행

In [7]:
conn = get_Redshift_connection(False)
cur = conn.cursor()

In [8]:
cur.execute("SELECT * FROM adhoc.yeojun_name_gender;")
res = cur.fetchall()
for r in res:
  print(r)

('Ben', 'Male')
('Maddie', 'Female')


In [9]:
cur.execute("DELETE FROM adhoc.yeojun_name_gender;")

In [ ]:
%%sql

select *
from adhoc.yeojun_name_gender;

In [11]:
cur.execute("SELECT * FROM adhoc.yeojun_name_gender;")
res = cur.fetchall()
for r in res:
  print(r)

In [12]:
cur.execute("INSERT INTO adhoc.yeojun_name_gender VALUES ('Keeyong', 'Male');")

In [ ]:
%%sql

select *
from adhoc.yeojun_name_gender;

In [ ]:
cur.execute("SELECT * FROM adhoc.yeojun_name_gender;")
res = cur.fetchall()
for r in res:
  print(r)

In [15]:
cur.execute("COMMIT;")   # conn.commit()는 동일한 결과를 가져옴. cur.execute("ROLLBACK;") conn.rollback()

In [ ]:
%%sql

select *
from adhoc.yeojun_name_gender;

In [17]:
conn.close()

### INSERT SQL을 autocommit=False로 실행하고 psycopg2로 컨트롤하기

In [18]:
conn = get_Redshift_connection(False)
cur = conn.cursor()

In [19]:
try:
  cur.execute("DELETE FROM adhoc.yeojun_name_gender;")
  cur.execute("INSERT INTO adhoc.yeojun_name_gender VALUES ('Claire', 'Female');")
  conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  conn.rollback()
finally :
  conn.close()

In [ ]:
%%sql

select *
from adhoc.yeojun_name_gender

### INSERT SQL을 autocommit=True로 실행하고 SQL로 컨트롤하기

In [26]:
conn = get_Redshift_connection(True)
cur = conn.cursor()

In [27]:
cur.execute("BEGIN;")
cur.execute("DELETE FROM adhoc.yeojun_name_gender;")
cur.execute("INSERT INTO adhoc.yeojun_name_gender VALUES ('Benjamin', 'Male');")
cur.execute("END;")

In [ ]:
%%sql

select *
from adhoc.yeojun_name_gender;

### 잘못된 SQL을 중간에 실행해보기

In [ ]:
cur.execute("BEGIN;")
cur.execute("DELETE FROM adhoc.yeojun_name_gender;")
cur.execute("INSERT INTO adhoc.yeojun_name_gender2 VALUES ('Andrew', 'Male');")
cur.execute("END;")

In [ ]:
%%sql

select *
from adhoc.yeojun_name_gender;

## 기타 고급 문법 소개

### EXCEPT 사용해보기 - 두 개의 SELECT 문의 결과 혹은 두 개의 테이블을 비교하는데 사용.
 * 기존 요약 테이블의 로직을 수정하는 경우 수정 전후를 비교하거나 하는데 많이 사용됨. QA용으로 아주 유용함
 * 각 SELECT문의 필드들의 수와 타입이 동일해야함
 * EXCEPT 대신 MINUS를 사용해도 됨

### UNION: 여러 SELECT 문을 합쳐서 하나의 SELECT를 생성 (합집합)

* 각 SELECT문의 필드들의 수와 타입이 동일해야함
* UNION과 UNION ALL의 차이점은?
  * UNION은 중복되는 레코드를 제거함
* 다른 소스에서 생긴 레코드들을 묶어서 새로운 테이블들을 만들 때 아주 유용
  * 예를 들면 물건 판매를 Shopify와 Amazon에 동시에 한다면 각 사이트에서 판매 레코드들을 UNION으로 묶어서 새로운 테이블을 생성가능 (CTAS)

In [31]:
%%sql

SELECT 'keeyong' as first_name, 'han' as last_name

UNION

SELECT 'elon', 'musk'

UNION

SELECT 'keeyong', 'han'

In [32]:
%%sql

SELECT 'keeyong' as first_name, 'han' as last_name

UNION ALL

SELECT 'elon', 'musk'

UNION ALL

SELECT 'keeyong', 'han'

### INTERSECT: SELECT 문들에서 공통적인 레코드를 돌려줌 (교집합)

### LISTAGG - GROUP BY Aggregate 함수 중의 하나

In [ ]:
%%sql

-- WITHIN GROUP (ORDER BY ts) channels: 순서를 결정
SELECT userid, LISTAGG(channel) WITHIN GROUP (ORDER BY ts) channels
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
GROUP BY 1
LIMIT 10;

In [ ]:
%%sql

SELECT userid, LISTAGG(channel, '->') WITHIN GROUP (ORDER BY ts) channels
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
GROUP BY 1
LIMIT 10;

In [ ]:
%%sql

SELECT usc.*, st.ts, LAG(channel, 1) OVER (PARTITION BY userId ORDER BY ts) prev_channel
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
ORDER BY usc.userid, st.ts
LIMIT 100;

In [ ]:
%%sql

SELECT usc.*, st.ts, LAG(channel, 1) OVER (PARTITION BY userId ORDER BY ts DESC) prev_channel
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
ORDER BY usc.userid, st.ts
LIMIT 100;

### JSON parsing 함수

In [ ]:
%%sql

SELECT JSON_EXTRACT_PATH_TEXT('{"f2":{"f3": "1"},"f4":{"f5":"99","f6":"star"}}','f4', 'f6');

In [ ]:
%%sql

SELECT JSON_EXTRACT_PATH_TEXT('{"f2":{"f3": "1"},"f4":{"f5":"99","f6":"star"}}','f4');